In [ ]:
# https://stackoverflow.com/questions/66360839/ipyleaflet-on-click-event-in-for-loop-calls-function-on-each-iteration

In [5]:
import ipywidgets as widgets
import pandas as pd
from IPython.display import display
from Magics import macro as magics
from Magics.macro import * 

from ecmwf.opendata import Client
import numpy as np
import ipyleaflet
from ipyleaflet import Map, Marker


import xarray as xr
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt

from bqplot import Lines, Figure, LinearScale, Axis
import bqplot

In [6]:
### PARAMETERS TO BE IMPORTED FROM CYCLONE SELECTION
initial_latlon = [25.9, -70.4]
initial_date = '20230830'
final_date = '20230903'

In [33]:
### FUNCTIONS DEF
def closest_point(point, points):
    """ 
        Find closest point from a list of points. 
        Input: 
            - point: reference point 
            - points: list of points 
        Output: 
            - closest point 
    """
    return points[cdist([point], points).argmin()]

def match_value(df, col1, x, col2):
    """ 
        Match value x from col1 row to value in col2.
        Input: 
            - df: dataframe
            - col1: string; column name 1 of the dataframe
            - x: pattern no match
            - col2: string; column name 2 of the dataframe
        Output: 
            - Value matched 
    """
    return df[df[col1] == x][col2].values

#def on_move(**kwargs):
def get_callback(data_allpoints):
    def on_move(**kwargs):
     #   global pos
        """
            Things that happen when the user moves the marker:
            1. The new position of the marker is readed. 
            2. For all variables, the values corresponding to the new position are obtained. 
            3. Lines of the plots (lines1.y/lines2.y/lines3.y/lines4.y) are updated with the values from (2). 

            Input: nothing
            Output: nothing
        """
        
        #pos = kwargs['location']
        #print(location)
       # pos = location
        return on_move
  #  pos = on_move()
  #  event, location = on_move()
    print('test: location')
    print(location)
   # data_plot = get_df_pos(data_allpoints, pos, steps_to_download)
   # lines1.y = data_plot['tp'].y.to_numpy()
   # lines2.y = data_plot['msl'].y.to_numpy()
   # lines3.y = data_plot['skt'].y.to_numpy()
   # lines4.y = data_plot['10fgg15'].y.to_numpy()
    

    
def get_allvars_allpoints(date_to_download):
    """
        Read grib file of each variable; transform it to pandas dataframe;
            store dataframes into a dictionary with the key being the variable names. 
        Input: 
            - date_to_download: string (YYYYMMDD) or integer (0/1/-1) indicating the 
                starting date of the forecast
        Output: 
            - out: dictionary containing the pandas dataframe of each variable
    """
    out = {}
    for var in ['tp', 'msl', 'skt', '10fgg15']:
        
        filename = 'data/' + var + '_' + str(date_to_download) + '.grib'
        ds = xr.open_dataset(filename, engine="cfgrib")

        df = ds.to_dataframe()
        lats = df.index.get_level_values("latitude")
        lons = df.index.get_level_values("longitude")
        
        if var == '10fgg15':
            vals = df['fg10g15']
        else:
            vals = df[var]
        # DATAFRAME OF ALL POINTS FROM MAP
        df_datapoints = pd.DataFrame({'Lat': lats, 'Lon':lons, 'Value': vals})
        df_datapoints['point'] = [(x, y) for x,y in zip(df_datapoints['Lat'], df_datapoints['Lon'])]
        out[var] = df_datapoints
    return(out)

def get_df_pos(data_allpoints, pos, steps_to_download):
    """
        For each variable, for a given point coordinate, returns the temporal evolution of the 
            variable in the given Lat, Lon coordinates.  
        Input: 
            - data_allpoints: dictionary containing a pandas dataframe on each key; key are the variable name
            - pos: lat lon tuple
        Output: 
            - dictionary containing a pandas dataframe for each variable
    """
    data_pos_allvars = {}
    for var in ['tp', 'msl', 'skt', '10fgg15']:

        df_datapoints = data_allpoints[var]
        
        df_selpoint = pd.DataFrame({'Lat': pd.Series(pos[0]), 'Lon':pd.Series(pos[1])})
        df_selpoint['point'] = [(x, y) for x,y in zip(df_selpoint['Lat'], df_selpoint['Lon'])]
        
        # IDENTIFY THE CLOSEST POINT
        df_selpoint['closest'] = [closest_point(x, list(df_datapoints['point'])) for x in df_selpoint['point']]

        # GET THE VALUE OF VAR FROM THE CLOSEST POINT FOR ALL TIMESTEPS:
        vec_values = [match_value(df_datapoints, 'point', x, 'Value') for x in df_selpoint['closest']]

        # RETURN DATAFRAME TO PLOT
        df_pos_var = pd.DataFrame({'x': pd.Series(steps_to_download), 'y':pd.Series(vec_values[0])})
        data_pos_allvars[var] = df_pos_var
    return(data_pos_allvars)

def get_initial_plot(data_allpoints, initial_latlon, steps_to_download):
    global lines1, lines2, lines3, lines4
    
    data_initial_plot = get_df_pos(data_allpoints, initial_latlon, steps_to_download)

    
    xdata1 = data_initial_plot['tp'].x.to_numpy()
    ydata1 = data_initial_plot['tp'].y.to_numpy()
    lines1 = bqplot.Bars(x=[xdata1], y=[ydata1], 
              scales={"x": LinearScale(min=float(min(xdata1)), max=float(max(xdata1))),
                      "y": LinearScale(min=0, max=float(max(ydata1)))})
    ax_y1 = Axis(label='mm',
                 scale=LinearScale(min=0, max=float(max(ydata1))), 
                 orientation="vertical", side="left")

    xdata2 = data_initial_plot['msl'].x.to_numpy()
    ydata2 = data_initial_plot['msl'].y.to_numpy()
    lines2 = Lines(x=[xdata2], y=[ydata2], 
              scales={"x": LinearScale(min=float(min(xdata2)), max=float(max(xdata2))),
                      "y": LinearScale(min=0, max=float(max(ydata2)))})
    ax_y2 = Axis(label='Pa', 
                 scale=LinearScale(min=0, max=float(max(ydata2))), 
                 orientation="vertical", side="left")
    
    
    xdata3 = data_initial_plot['skt'].x.to_numpy()
    ydata3 = data_initial_plot['skt'].y.to_numpy()
    lines3 = Lines(x=[xdata3], y=[ydata3], 
              scales={"x": LinearScale(min=float(min(xdata3)), max=float(max(xdata3))),
                      "y": LinearScale(min=0, max=float(max(ydata3)))})
    ax_y3 = Axis(label='K', 
                 scale=LinearScale(min=0, max=float(max(ydata3))), 
                 orientation="vertical", side="left")
    
    xdata4 = data_initial_plot['10fgg15'].x.to_numpy()
    ydata4 = data_initial_plot['10fgg15'].y.to_numpy()
    lines4 = Lines(x=[xdata4], y=[ydata4], 
              scales={"x": LinearScale(min=float(min(xdata4)), max=float(max(xdata4))),
                      "y": LinearScale(min=0, max=float(max(ydata4)))})
    ax_y4 = Axis(label='%', 
                scale=LinearScale(min=0, max=float(max(ydata4))), 
                 orientation="vertical", side="left")

    ax_x = Axis(label="Forecasted hours",
            scale=LinearScale(min=float(min(xdata1)), max=float(max(xdata1))),
            num_ticks=4)

    p1 = Figure(
            axes=[ax_x, ax_y1],
            title='Total precipitation',
            marks=[lines1],
            animation_duration=10,
            layout={"max_width": "200px", "max_height": "200px"},
        )

    p2 = Figure(
            axes=[ax_x, ax_y2], title='Mean sea level pressure',
            marks=[lines2], animation_duration=10,
            layout={"max_width": "200px", "max_height": "200px"},
        )

    p3 = Figure(
            axes=[ax_x, ax_y3], title='Skin temperature',
            marks=[lines3], animation_duration=10,
            layout={"max_width": "200px", "max_height": "200px"},
        )

    p4 = Figure(
            axes=[ax_x, ax_y4],title='Wind gust 10m > 15m/s',
            marks=[lines4], animation_duration=10,
            layout={"max_width": "200px", "max_height": "200px"},
        )

    return p1, p2, p3, p4

def download_data_s5(date_to_download, steps_to_download):
    ### DOWNLOAD DATA
    print('Downloading data...')
    client = Client("azure", beta=True) # ecwf: last five days
    for var in ['tp', 'msl', 'skt']:
        print(var)
        client.retrieve(
            date = date_to_download, #date start of the forecast
            time = 0,  # time start of the forecast or 12
            step = steps_to_download, #step of the forecast
            stream = "oper",
            type = "fc",
            levtype = "sfc",
            param = var,
            target = 'data/' + var + '_' + str(date_to_download) + '.grib',
        )

    var = '10fgg15'
    client.retrieve(
            date = date_to_download, #date start of the forecast
            time = 0,  # time start of the forecast or 12
            step = '216-240', #step of the forecast
            stream = "enfo",
            type = "ep",
            param = var,
            target = 'data/' + var + '_' + str(date_to_download) + '.grib',
        )    

def load_data_s5(date_to_download, filename_avg_track):
    global data_allpoints
    print('Loading data...')
    grib_data = {}
    for var in ['tp', 'msl', 'skt', '10fgg15']:

        filename = 'data/' + var + '_' + str(date_to_download) + '.grib'
        grib_data[var] = mgrib(grib_input_file_name=filename)

    data_allpoints = get_allvars_allpoints(date_to_download)
    # LOAD AVG TRACK
    df_avg_track = pd.read_csv(filename_avg_track)
    return(data_allpoints, df_avg_track)

# CREATE MAP 

def map_s5(initial_latlon, initial_date, final_date, filename_avg_track):
    global steps_to_download
    ### VARIABLE INITIALIZATION
    cyclone_days = pd.date_range(start=initial_date, end=final_date)
    date_to_download = initial_date
    steps_to_download = list(np.arange(0,240, 12)[0:2*len(cyclone_days)])
    # download data: 
    download_data_s5(date_to_download, steps_to_download)

    ### LOAD DATA
    data_allpoints, df_avg_track =load_data_s5(date_to_download, filename_avg_track)
    print('Printing map...')
    m = Map(
        center=[initial_latlon[0], initial_latlon[1]+50],
        basemap=ipyleaflet.basemaps.OpenStreetMap.France,
        zoom = 2,
    )

    p1, p2, p3, p4 = get_initial_plot(data_allpoints, initial_latlon, steps_to_download)
    main_figure = widgets.VBox([widgets.HBox([p1,p2]), widgets.HBox([p3,p4])], 
                             align_content = 'stretch')
    main_figure.layout.width='400px'
    main_figure.layout.height='400px'
    widget_control1 = ipyleaflet.WidgetControl(widget=main_figure, position="bottomright")

    marker = Marker(location=initial_latlon, draggable=True, name = 'Position') 
    m.add_layer(marker)
    marker.on_move(get_callback(data_allpoints))


    m.add(widget_control1)
    avg_track = ipyleaflet.AntPath(locations = df_avg_track.values.tolist(), color = "red")
    m.add_layer(avg_track)
    return(m)

In [34]:
maps5 = map_s5(initial_latlon, initial_date, final_date, 'df_avg_track.csv')

tp


<multiple>:   0%|          | 0.00/3.42M [00:00<?, ?B/s]

msl


<multiple>:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

skt


...

Ignoring index file 'data/tp_20230830.grib.923a8.idx' older than GRIB file


Loading data...


Ignoring index file 'data/msl_20230830.grib.923a8.idx' older than GRIB file
Ignoring index file 'data/skt_20230830.grib.923a8.idx' older than GRIB file
Ignoring index file 'data/10fgg15_20230830.grib.923a8.idx' older than GRIB file


Printing map...
prova: location


NameError: name 'location' is not defined

In [8]:
display(maps5)

Map(center=[25.9, -20.400000000000006], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_ti…

[25.799891182088334, -54.488951402987794]
[7.36246686553575, -67.50266079087672]
[21.289374355860424, -114.61901137776977]
[37.99616267972814, -124.80677298474123]
[37.71859032558816, -93.15759030597556]
[42.553080288955826, -39.01236285689152]
[31.653381399664, -56.605158163346914]
[6.315298538330033, -61.524444285737154]
[30.751277776257812, -48.16132979585635]
[14.604847155053898, -52.7353098724702]
[17.308687886770034, -76.99714951541876]
